In [255]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import  when,mean ,lit,ceil
# Création d'une session Spark
spark = SparkSession.builder\
         .appName("RatingEngine")\
         .config("spark.jars", r".conf\postgresql-42.7.6.jar")\
         .config("spark.driver.extraClassPath", r".\conf\postgresql-42.7.6.jar")\
         .config("spark.executor.extraClassPath", r".\conf\\postgresql-42.7.6.jar")\
         .config("spark.sql.shuffle.partitions", "50")\
         .config("spark.sql.autoBroadcastJoinThreshold", "10485760")\
         .config("spark.driver.memory", "6g")\
         .getOrCreate()




In [256]:
data_path = r".\output_dir" 
cleanedCdrsDf  = spark.read.format("parquet") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(data_path)

In [257]:
customersDf = spark.read \
    .format("jdbc")\
    .option("url",      "jdbc:postgresql://localhost:5432/projet_spark")\
    .option("dbtable",  "customers")\
    .option("user",     "postgres")\
    .option("password", "0000")\
    .load() 

customersDf.show(5)


+------------+-----------------+-----------------+------------+---------------+------+--------------------+
| customer_id|    customer_name|subscription_type|rate_plan_id|activation_date|status|              region|
+------------+-----------------+-----------------+------------+---------------+------+--------------------+
|212768116861|    Anwar Kettani|         postpaid|      PLAN_C|     2024-04-29|active|Béni Mellal-Khénifra|
|212760204113|Naoual El Othmani|         postpaid|      PLAN_A|     2024-12-07|active|      Drâa-Tafilalet|
|212704344822|     Ehab Ouchrif|         postpaid|      PLAN_A|     2023-07-29|active|Laâyoune-Sakia El...|
|212658901536|     Selma Msaddi|         postpaid|      PLAN_C|     2023-06-16|active|         Souss-Massa|
|212699900093| Brahim El Hilali|         postpaid|      PLAN_A|     2024-07-27|active|Dakhla-Oued Ed Dahab|
+------------+-----------------+-----------------+------------+---------------+------+--------------------+
only showing top 5 rows



In [258]:
catalogDf = spark.read \
    .option("header", True) \
    .csv("ressources/product_catalog.csv")


In [259]:
from pyspark.sql.functions import col

ratePlansDf = spark.read \
    .option("header", True) \
    .csv("ressources/rate_plans.csv") \
    .withColumn("unit_price", col("unit_price").cast("double")) \
    .withColumn("free_units", col("free_units").cast("int")) \
    .withColumn("tier_threshold", col("tier_threshold").cast("int")) \
    .withColumn("tier_price", col("tier_price").cast("double"))


In [260]:
cdrs = (cleanedCdrsDf
        .withColumn(
            "customer_id",
            when(col("record_type") == "voice", col("caller_id"))
            .when(col("record_type") == "sms",  col("sender_id"))
            .otherwise(col("user_id"))
        ))

In [261]:
# ─── jointure client → plan ─────────────────────────────────────────────
cdrs1 = cdrs.join(
    customersDf.filter((col("status") == "active") &
                       (col("subscription_type") == "postpaid"))
               .select("customer_id", "rate_plan_id"),
    on="customer_id",
    how="inner"
)
cdrs1.show()

+------------+--------------------+-------------------+-------------+----------+------------+------------+------------+------------+-------------+------------+------------+-----------+------------+--------------------+--------------+--------+-----------+------------+
| customer_id|           record_ID|          timestamp|      cell_id|technology|   caller_id|   callee_id|duration_sec|product_code|rating_status|   sender_id| receiver_id|receiver_cc|     user_id|session_duration_sec|data_volume_mb|batch_id|record_type|rate_plan_id|
+------------+--------------------+-------------------+-------------+----------+------------+------------+------------+------------+-------------+------------+------------+-----------+------------+--------------------+--------------+--------+-----------+------------+
|212768116861|30fcd930-6ec0-4e6...|2025-06-01 14:55:06|       FES_02|        4G|        NULL|        NULL|        NULL|  DATA_BASIC|        ready|        NULL|        NULL|       NULL|212768116861

In [262]:
# ─── jointure produit (rate_type, service_type) ────────────────────────
cdrs2 = cdrs1.join(catalogDf, on="product_code", how="left")
cdrs2.show()

+------------+------------+--------------------+-------------------+-------------+----------+------------+------------+------------+-------------+------------+------------+-----------+------------+--------------------+--------------+--------+-----------+------------+------------+------+---------+--------------------+
|product_code| customer_id|           record_ID|          timestamp|      cell_id|technology|   caller_id|   callee_id|duration_sec|rating_status|   sender_id| receiver_id|receiver_cc|     user_id|session_duration_sec|data_volume_mb|batch_id|record_type|rate_plan_id|service_type|  unit|rate_type|         description|
+------------+------------+--------------------+-------------------+-------------+----------+------------+------------+------------+-------------+------------+------------+-----------+------------+--------------------+--------------+--------+-----------+------------+------------+------+---------+--------------------+
|  DATA_BASIC|212768116861|30fcd930-6ec0-4e

In [263]:
cdrs3 = cdrs2.join(ratePlansDf,
                   on=["rate_plan_id", "product_code"],
                   how="left")

In [264]:
cdrs3.show()

+------------+------------+------------+--------------------+-------------------+-------------+----------+------------+------------+------------+-------------+------------+------------+-----------+------------+--------------------+--------------+--------+-----------+------------+------+---------+--------------------+------------+----------+----------+--------------+----------+
|rate_plan_id|product_code| customer_id|           record_ID|          timestamp|      cell_id|technology|   caller_id|   callee_id|duration_sec|rating_status|   sender_id| receiver_id|receiver_cc|     user_id|session_duration_sec|data_volume_mb|batch_id|record_type|service_type|  unit|rate_type|         description|service_type|unit_price|free_units|tier_threshold|tier_price|
+------------+------------+------------+--------------------+-------------------+-------------+----------+------------+------------+------------+-------------+------------+------------+-----------+------------+--------------------+---------

In [265]:
cdrs3=cdrs3.drop("service_type")
cdrs3.show()

+------------+------------+------------+--------------------+-------------------+-------------+----------+------------+------------+------------+-------------+------------+------------+-----------+------------+--------------------+--------------+--------+-----------+------+---------+--------------------+----------+----------+--------------+----------+
|rate_plan_id|product_code| customer_id|           record_ID|          timestamp|      cell_id|technology|   caller_id|   callee_id|duration_sec|rating_status|   sender_id| receiver_id|receiver_cc|     user_id|session_duration_sec|data_volume_mb|batch_id|record_type|  unit|rate_type|         description|unit_price|free_units|tier_threshold|tier_price|
+------------+------------+------------+--------------------+-------------------+-------------+----------+------------+------------+------------+-------------+------------+------------+-----------+------------+--------------------+--------------+--------+-----------+------+---------+--------

In [266]:
from pyspark.sql.functions import col, min as spark_min

agg = (cdrs3
       .filter(col("record_type") == "data")
       .groupBy("customer_id")
       .agg(
           spark_min("data_volume_mb")
           .alias("user_min_mb")
       )
)

cdrs4 = cdrs3.join(agg, on="customer_id", how="left")

In [267]:
cdrs4.show()

+------------+------------+------------+--------------------+-------------------+-------------+----------+------------+------------+------------+-------------+------------+------------+-----------+------------+--------------------+--------------+--------+-----------+------+---------+--------------------+----------+----------+--------------+----------+-----------+
| customer_id|rate_plan_id|product_code|           record_ID|          timestamp|      cell_id|technology|   caller_id|   callee_id|duration_sec|rating_status|   sender_id| receiver_id|receiver_cc|     user_id|session_duration_sec|data_volume_mb|batch_id|record_type|  unit|rate_type|         description|unit_price|free_units|tier_threshold|tier_price|user_min_mb|
+------------+------------+------------+--------------------+-------------------+-------------+----------+------------+------------+------------+-------------+------------+------------+-----------+------------+--------------------+--------------+--------+-----------+-

In [268]:
cdrs4 = cdrs4.withColumn(
    "data_volume_mb",
    when( (col("data_volume_mb")=="inconnu"), col("user_min_mb"))
    .otherwise(col("data_volume_mb"))
)

In [269]:
cdrs4 = cdrs4.withColumn(
    "data_volume_mb",
    when( ((col("user_min_mb")=="inconnu") &(col("data_volume_mb")=="inconnu")), lit(0)).otherwise(col("data_volume_mb")))

In [270]:
cdrs4 = cdrs4.withColumn(
    "rating_status",
    when((col("rating_status")=="needs_review")&(col("data_volume_mb")=="inconnu"), "unmatched").otherwise(col("rating_status"))
)

In [271]:
cdrs4 = cdrs4.withColumn(
    "rating_status",
    when((col("rating_status")=="needs_review"), "ready").otherwise(col("rating_status"))
)

In [272]:
cdrsss=cdrs4.filter(cdrs4.rating_status == "ready")
cdrsss.show()

+------------+------------+------------+--------------------+-------------------+-------------+----------+------------+------------+------------+-------------+------------+-----------+-----------+------------+--------------------+--------------+--------+-----------+------+---------+--------------------+----------+----------+--------------+----------+-----------+
| customer_id|rate_plan_id|product_code|           record_ID|          timestamp|      cell_id|technology|   caller_id|   callee_id|duration_sec|rating_status|   sender_id|receiver_id|receiver_cc|     user_id|session_duration_sec|data_volume_mb|batch_id|record_type|  unit|rate_type|         description|unit_price|free_units|tier_threshold|tier_price|user_min_mb|
+------------+------------+------------+--------------------+-------------------+-------------+----------+------------+------------+------------+-------------+------------+-----------+-----------+------------+--------------------+--------------+--------+-----------+----

In [273]:
cdrs4.show()

+------------+------------+------------+--------------------+-------------------+-------------+----------+------------+------------+------------+-------------+------------+-----------+-----------+------------+--------------------+--------------+--------+-----------+------+---------+--------------------+----------+----------+--------------+----------+-----------+
| customer_id|rate_plan_id|product_code|           record_ID|          timestamp|      cell_id|technology|   caller_id|   callee_id|duration_sec|rating_status|   sender_id|receiver_id|receiver_cc|     user_id|session_duration_sec|data_volume_mb|batch_id|record_type|  unit|rate_type|         description|unit_price|free_units|tier_threshold|tier_price|user_min_mb|
+------------+------------+------------+--------------------+-------------------+-------------+----------+------------+------------+------------+-------------+------------+-----------+-----------+------------+--------------------+--------------+--------+-----------+----

In [274]:
cdrs4 = cdrs4.withColumn(
    "session_duration_sec",
    when((col("session_duration_sec").isNull() | (col("session_duration_sec") < 0))&(col("record_type")=="data"), lit(0))
    .otherwise(col("session_duration_sec"))
)

In [275]:
cdrs4 = cdrs4.withColumn(
    "billable_units",
    when(col("record_type") == "voice",
         ceil(col("duration_sec") / 60))       
    .when(col("record_type") == "sms",
         lit(1))
    .otherwise(                               # data
         col("data_volume_mb"))
)


In [276]:
cdrs4.show()

+------------+------------+------------+--------------------+-------------------+-------------+----------+------------+------------+------------+-------------+------------+-----------+-----------+------------+--------------------+--------------+--------+-----------+------+---------+--------------------+----------+----------+--------------+----------+-----------+--------------+
| customer_id|rate_plan_id|product_code|           record_ID|          timestamp|      cell_id|technology|   caller_id|   callee_id|duration_sec|rating_status|   sender_id|receiver_id|receiver_cc|     user_id|session_duration_sec|data_volume_mb|batch_id|record_type|  unit|rate_type|         description|unit_price|free_units|tier_threshold|tier_price|user_min_mb|billable_units|
+------------+------------+------------+--------------------+-------------------+-------------+----------+------------+------------+------------+-------------+------------+-----------+-----------+------------+--------------------+----------

In [277]:
from pyspark.sql.functions import coalesce

cdrs4 = (cdrs4
    .withColumn("free_units",    coalesce(col("free_units"),    lit(0)))
    .withColumn("tier_threshold",coalesce(col("tier_threshold"),lit(0)))
    .withColumn("unit_price",    coalesce(col("unit_price"),    lit(0.0)))
    .withColumn("tier_price",    coalesce(col("tier_price"),    lit(0.0)))
)


In [278]:
cdrsRated = cdrs4.withColumn(
    "cost",
    when(col("rate_type") == "flat",
         when(col("billable_units") <= col("free_units"), lit(0))
         .otherwise((col("billable_units") - col("free_units")) * col("unit_price")))
    .when(col("rate_type") == "tiered",
         when(col("billable_units") <= col("free_units"), lit(0))
         .when(col("billable_units") <= col("tier_threshold"),
               (col("billable_units") - col("free_units")) * col("unit_price"))
         .otherwise(
               (col("tier_threshold") - col("free_units")) * col("unit_price") +
               (col("billable_units") - col("tier_threshold")) * col("tier_price")))
)

cdrsRated = cdrsRated.withColumn(
    "rating_status",
    when(col("cost").isNull(), "unmatched")
    .otherwise("rated")
)


In [279]:
cdrsRated.select("record_ID","customer_id","product_code",
                 "billable_units","unit_price","cost","rating_status") \
         .show(truncate=False)


+------------------------------------+------------+------------+--------------+----------+--------------------+-------------+
|record_ID                           |customer_id |product_code|billable_units|unit_price|cost                |rating_status|
+------------------------------------+------------+------------+--------------+----------+--------------------+-------------+
|f6911324-9ca7-43b6-b17b-bc98deb5f085|212668884253|DATA_PREMIUM|1889.31       |0.015     |57.8755             |rated        |
|f988375f-7a1a-484c-9295-84252e696400|212619280249|DATA_BASIC  |207.05        |0.022     |4.0051              |rated        |
|6c8b6730-6d33-4471-be29-44390d762e69|212619280249|VOICE_NAT   |9             |0.012     |0.036000000000000004|rated        |
|5fbc4373-b04e-4d91-9691-ea790928ba63|212736864154|VOICE_NAT   |1             |0.01      |0.0                 |rated        |
|514b5909-5061-4712-be2e-55cd7d38edcb|212774638478|DATA_BASIC  |0             |0.018     |0.0                 |rated  

In [280]:
from pyspark.sql.functions import date_format

cdrsRated = cdrsRated.withColumn(
    "billing_period",
    date_format(col("timestamp"), "yyyy-MM")
)
cdrsRated=cdrsRated.drop("user_min_mb")

In [281]:
(
 cdrsRated
 .write
 .mode("overwrite")
 .partitionBy("billing_period")
 .parquet("rated_cdrs/")
)

print("✓ Rating terminé : Parquet écrit dans rated_cdrs/")


✓ Rating terminé : Parquet écrit dans rated_cdrs/


In [282]:
cdrsRated.show()

+------------+------------+------------+--------------------+-------------------+-------------+----------+------------+------------+------------+-------------+------------+-----------+-----------+------------+--------------------+--------------+--------+-----------+------+---------+--------------------+----------+----------+--------------+----------+--------------+--------------------+--------------+
| customer_id|rate_plan_id|product_code|           record_ID|          timestamp|      cell_id|technology|   caller_id|   callee_id|duration_sec|rating_status|   sender_id|receiver_id|receiver_cc|     user_id|session_duration_sec|data_volume_mb|batch_id|record_type|  unit|rate_type|         description|unit_price|free_units|tier_threshold|tier_price|billable_units|                cost|billing_period|
+------------+------------+------------+--------------------+-------------------+-------------+----------+------------+------------+------------+-------------+------------+-----------+--------